In [6]:
import pandas as pd
import numpy as np
import joblib

# Model / Scaler
xgb_model = joblib.load('../src/models/hypertension_xgb.pkl')
scaler = joblib.load('../src/models/num_scaler.pkl')

# Numeric features to scale
NUM_COLS = ['salt_intake', 'sleep_duration', 'bmi']

def predict_patient(raw_patient: dict):
    """
    raw_patient example:
    {
        'age': 45,
        'salt_intake': 8.5,
        'stress_score': 6,
        'sleep_duration': 6,
        'bmi': 31,
        'bp_history': 'Prehypertension',
        'family_history': 'Yes',
        'exercise_level': 'Moderate',
        'smoking_status': 'Non-Smoker'
    }
    """

    # 1. Converting to dataframe 
    df = pd.DataFrame([raw_patient])

    # 2. Ordinal encoding (as in dataset)
    df['bp_history'] = df['bp_history'].map({
        'Normal': 0,
        'Hypertension': 1,
        'Prehypertension': 2
    })
    df['family_history'] = df['family_history'].map({'No': 0, 'Yes': 1})
    df['exercise_level'] = df['exercise_level'].map({'Low': 0, 'Moderate': 1, 'High': 2})
    df['smoking_status'] = df['smoking_status'].map({'Non-Smoker': 0, 'Smoker': 1})

    # 3. Numeric features scaling
    df[NUM_COLS] = scaler.transform(df[NUM_COLS])

    # 4. Categorial features encoding (as in XGBoost learning)
    df_encoded = pd.get_dummies(df)

    df_encoded = df_encoded.reindex(
        columns=xgb_model.feature_names_in_,
        fill_value=0
    )

    # 5. Results as prediction and probability of positive class.
    pred = xgb_model.predict(df_encoded)[0]
    prob = xgb_model.predict_proba(df_encoded)[0, 1]

    return pred, prob

In [7]:
patient = {
    'age': 20,
    'salt_intake': 10,
    'stress_score': 5,
    'sleep_duration': 5,
    'bmi': 19.4,
    'bp_history': 'Hypertension',
    'family_history': 'Yes',
    'exercise_level': 'Low',
    'smoking_status': 'Smoker'
}

pred, prob = predict_patient(patient)

print(f'XGBoost Prediction: {"risk of hypertension" if pred==1 else "no risk of hypertension"}')
print(f'Hypertension probability: {int(prob * 100)}%')

XGBoost Prediction: no risk of hypertension
Hypertension probability: 33%
